In [27]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.test
dbcol_review = db.review
dbcol_detail = db.ani_detail

In [29]:
from surprise import Dataset
from surprise import Reader

from collections import defaultdict
import numpy as np
import pandas as pd

# 애니 정보 불러오기
ani_df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1}))
ani_df.columns = ["_id", "animation", "ani_name"]

# 평가 데이터 불러오기
profile = list(dbcol_review.find({}))
rating_df = pd.DataFrame(profile)
rating_df = rating_df[["animation", "profile", "score"]]
rating_df["profile"] = rating_df["profile"].map(lambda x:x["id"])
rating_df.columns = ["animation", "user_id", "score"]

ani_ratings = pd.merge(ani_df, rating_df, left_on='animation', right_on='animation')

# 애니를 중심으로 평점을 보기 위한 데이터 프레임 생성
ani_rating_summary = ani_ratings[["animation", "score"]].groupby(["animation"]).agg(["count", "median", "mean"])
# score 안에 count, median, mean으로 멀티 인덱스이므로 수정
ani_rating_summary.columns = ["count", "median", "mean"]

df = pd.DataFrame(ani_ratings, columns=['user_id', 'animation', 'score'])

In [30]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

# 데이터 변환
# reader = Reader(rating_scale=(0.5, 5))
# data = Dataset.load_from_df(df, reader)
reader = Reader(line_format = 'user item rating', rating_scale=(0.5,5))
data = Dataset.load_from_df(df[['user_id', 'animation', 'score']], reader)

# 훈련 및 테스트 데이터 분리
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
 
# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)
 
# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
for i in range(10):
    print(predictions[i])

accuracy.rmse(predictions)

user: 3234638    item: 13395      r_ui = 0.00   est = 5.00   {'was_impossible': False}
user: 605066     item: 15980      r_ui = 5.00   est = 5.00   {'was_impossible': False}
user: 3598578    item: 39990      r_ui = 0.00   est = 5.00   {'was_impossible': False}
user: 218403     item: 40372      r_ui = 0.00   est = 5.00   {'was_impossible': False}
user: 433237     item: 40165      r_ui = 4.50   est = 5.00   {'was_impossible': False}
user: 21875      item: 40695      r_ui = 3.00   est = 5.00   {'was_impossible': False}
user: 163676     item: 33234      r_ui = 2.00   est = 5.00   {'was_impossible': False}
user: 1153896    item: 39499      r_ui = 5.00   est = 5.00   {'was_impossible': False}
user: 1864254    item: 23698      r_ui = 0.00   est = 5.00   {'was_impossible': False}
user: 2367974    item: 38106      r_ui = 3.50   est = 5.00   {'was_impossible': False}
RMSE: 2.8340


2.8340215695705155

In [31]:
print(len(predictions))
for i in range(98070):
    if predictions[i][3] != 5:
        print(predictions[i])
#     print(predictions[i][2], predictions[i][3])

98072


zero 제거

In [32]:
from surprise import Dataset
from surprise import Reader

from collections import defaultdict
import numpy as np
import pandas as pd

# 애니 정보 불러오기
ani_df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1}))
ani_df.columns = ["_id", "animation", "ani_name"]

# 평가 데이터 불러오기
profile = list(dbcol_review.find({}))
rating_df = pd.DataFrame(profile)
rating_df = rating_df[["animation", "profile", "score"]]
rating_df["profile"] = rating_df["profile"].map(lambda x:x["id"])
rating_df.columns = ["animation", "user_id", "score"]
drop_index = rating_df[rating_df['score'] == 0].index
rating_df.drop(drop_index, inplace=True)

ani_ratings = pd.merge(ani_df, rating_df, left_on='animation', right_on='animation')

# 애니를 중심으로 평점을 보기 위한 데이터 프레임 생성
ani_rating_summary = ani_ratings[["animation", "score"]].groupby(["animation"]).agg(["count", "median", "mean"])
# score 안에 count, median, mean으로 멀티 인덱스이므로 수정
ani_rating_summary.columns = ["count", "median", "mean"]

df = pd.DataFrame(ani_ratings, columns=['user_id', 'animation', 'score'])

In [33]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

# 데이터 변환
# reader = Reader(rating_scale=(0.5, 5))
# data = Dataset.load_from_df(df, reader)
reader = Reader(line_format = 'user item rating', rating_scale=(0.5,5))
data = Dataset.load_from_df(df[['user_id', 'animation', 'score']], reader)

# 훈련 및 테스트 데이터 분리
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
 
# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)
 
# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
for i in range(10):
    print(predictions[i])

accuracy.rmse(predictions)

user: 3696467    item: 40697      r_ui = 4.00   est = 5.00   {'was_impossible': False}
user: 900431     item: 23056      r_ui = 5.00   est = 5.00   {'was_impossible': False}
user: 2734490    item: 39367      r_ui = 2.50   est = 5.00   {'was_impossible': False}
user: 2509539    item: 39847      r_ui = 5.00   est = 5.00   {'was_impossible': False}
user: 2617787    item: 39985      r_ui = 5.00   est = 5.00   {'was_impossible': False}
user: 337571     item: 39081      r_ui = 4.00   est = 5.00   {'was_impossible': False}
user: 2206431    item: 39192      r_ui = 5.00   est = 5.00   {'was_impossible': False}
user: 1146059    item: 39887      r_ui = 0.50   est = 5.00   {'was_impossible': False}
user: 1200342    item: 37189      r_ui = 0.50   est = 5.00   {'was_impossible': False}
user: 3347208    item: 39708      r_ui = 5.00   est = 5.00   {'was_impossible': False}
RMSE: nan


nan

In [35]:
print(len(predictions))
for i in range(71057):
    if predictions[i][3] != 5:
        print(predictions[i])
#     print(predictions[i][2], predictions[i][3])

71057
